## 🐍 시계열 데이터 분석 기초 (Time Series Analysis Basics)

이 튜토리얼에서는 시간의 흐름에 따라 기록된 데이터, 즉 **시계열(Time Series)** 데이터의 기본적인 개념과 분석 기법을 다룹니다. 시계열 데이터가 가진 고유한 특성들을 분해해보고, 데이터를 부드럽게 만드는 평활(Smoothing) 기법을 거쳐 미래를 예측하는 모델링까지 단계별로 학습합니다.

### 초기 설정: 라이브러리 설치 및 임포트

In [ ]:
# !pip install plotly statsmodels koreanize-matplotlib

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
import koreanize_matplotlib

# Pandas가 Plotly를 백엔드로 사용하도록 설정
pd.options.plotting.backend = "plotly"

---

## 1. 시계열 데이터의 이해와 분해 (Understanding and Decomposing Time Series)

### 💡 개념 (Concept)

시계열 데이터는 일반적인 데이터와 달리 '시간'이라는 축을 기준으로 정보가 나열되어 있어, 시간의 흐름에 따른 특정 패턴을 가집니다. 이러한 패턴은 주로 다음 세 가지 요소로 구성됩니다.

* **추세 (Trend, $T_t$)**: 데이터가 장기적으로 증가하거나 감소하는 방향성. 예를 들어, 회사의 연간 매출액은 꾸준히 증가하는 추세를 보일 수 있습니다.
* **계절성 (Seasonality, $S_t$)**: 특정 시간 주기로 반복되는 패턴. 예를 들어, 아이스크림 판매량은 매년 여름마다 급증하는 계절성을 가집니다.
* **잔차 (Residual, $E_t$)**: 추세와 계절성으로 설명할 수 없는, 불규칙하고 예측 불가능한 변동. 노이즈(Noise) 또는 오차(Error)라고도 합니다.

이 세 가지 요소를 분리해내는 것을 **시계열 분해(Time Series Decomposition)**라고 하며, 이를 통해 데이터의 구조를 더 깊이 이해할 수 있습니다. 분해 모델은 크게 두 가지가 있습니다.

* **가법 모델 (Additive Model)**: $Y_t = T_t + S_t + E_t$. 계절성의 변동폭이 데이터의 크기와 상관없이 일정할 때 사용합니다.
* **승법 모델 (Multiplicative Model)**: $Y_t = T_t \times S_t \times E_t$. 계절성의 변동폭이 데이터의 크기에 비례하여 커지거나 작아질 때 사용합니다. (예: 연도별 항공기 탑승객 수)

### 💻 예시 코드 (Example Code)

국제 항공기 탑승객 수는 시간이 지남에 따라 전반적으로 증가(추세)하고, 매년 여름 휴가철에 탑승객이 급증하는 패턴(계절성)을 보입니다. 또한, 계절성의 변동폭은 탑승객 수 자체가 증가함에 따라 함께 커지는 경향이 있으므로 **승법 모델(multiplicative model)**을 사용하는 것이 적합합니다.

In [ ]:
# 1. 데이터 로드 및 준비
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
df_air = pd.read_csv(url, index_col='Month', parse_dates=True)
df_air.rename(columns={'Passengers': '탑승객 수'}, inplace=True)

# 2. 원본 데이터 시각화
fig = px.line(df_air, x=df_air.index, y='탑승객 수', title='월별 국제 항공기 탑승객 수 (1949-1960)')
fig.show()

# 3. 승법 모델을 이용한 시계열 분해
result = seasonal_decompose(df_air['탑승객 수'], model='multiplicative')

# 4. 분해 결과 시각화
decomposed_fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    subplot_titles=("Observed (원본)", "Trend (추세)", "Seasonal (계절성)", "Residual (잔차)")
)

decomposed_fig.add_trace(go.Scatter(x=result.observed.index, y=result.observed, mode='lines', name='Observed'), row=1, col=1)
decomposed_fig.add_trace(go.Scatter(x=result.trend.index, y=result.trend, mode='lines', name='Trend'), row=2, col=1)
decomposed_fig.add_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal, mode='lines', name='Seasonal'), row=3, col=1)
decomposed_fig.add_trace(go.Scatter(x=result.resid.index, y=result.resid, mode='lines', name='Residual'), row=4, col=1)

decomposed_fig.update_layout(height=700, title_text="항공기 탑승객 데이터 시계열 분해 (승법 모델)", showlegend=False)
decomposed_fig.show()

### ✏️ 연습 문제 (Practice Problems)

1.  아래 URL은 1980년부터 1990년까지 호주 멜버른의 일일 최저 기온을 기록한 데이터입니다. 이 데이터를 `pandas`로 불러와 시계열 데이터로 준비하세요.
    * 데이터 URL: `https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv`
    * 'Date' 컬럼을 인덱스로 만들고, 날짜 형식으로 변환하세요 (`parse_dates=True`).
    * 'Temp' 컬럼의 이름을 '최저기온'으로 변경하세요.

2.  준비된 멜버른 기온 데이터를 선 그래프로 시각화하여 전체적인 패턴을 파악해 보세요.

3.  멜버른의 일일 최저 기온 데이터는 계절적 변동폭이 수준과 관계없이 비교적 일정해 보입니다. `seasonal_decompose` 함수를 사용하여 **가법 모델(additive model)**로 시계열을 분해하고, 그 결과를 시각화하여 추세, 계절성, 잔차를 확인해 보세요. (계절성 주기는 1년, 즉 365일로 설정해 보세요: `period=365`)

In [3]:
# 연습 문제 1, 2, 3번 풀이 공간
# 연습 문제 1: 멜버른 기온 데이터 로드 및 준비
url_temp = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
df_temp = pd.read_csv(url_temp, index_col='Date', parse_dates=True)
df_temp.rename(columns={'Temp': '최저기온'}, inplace=True)

print("멜버른 기온 데이터 정보:")
print(df_temp.info())
print("\n데이터 미리보기:")
print(df_temp.head())

멜버른 기온 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3650 entries, 1981-01-01 to 1990-12-31
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   최저기온    3650 non-null   float64
dtypes: float64(1)
memory usage: 57.0 KB
None

데이터 미리보기:
            최저기온
Date            
1981-01-01  20.7
1981-01-02  17.9
1981-01-03  18.8
1981-01-04  14.6
1981-01-05  15.8


In [4]:

# 연습 문제 2: 멜버른 기온 데이터 시각화
fig_temp = px.line(df_temp, x=df_temp.index, y='최저기온', 
                   title='멜버른 일일 최저 기온 (1981-1990)')
fig_temp.update_layout(xaxis_title='날짜', yaxis_title='최저기온 (°C)')
fig_temp.show()


In [5]:
# 연습 문제 3: 가법 모델을 이용한 시계열 분해
result_temp = seasonal_decompose(df_temp['최저기온'], model='additive', period=365)

# 분해 결과 시각화
decomposed_fig_temp = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    subplot_titles=("Observed (원본)", "Trend (추세)", "Seasonal (계절성)", "Residual (잔차)")
)

decomposed_fig_temp.add_trace(go.Scatter(x=result_temp.observed.index, y=result_temp.observed, 
                                        mode='lines', name='Observed'), row=1, col=1)
decomposed_fig_temp.add_trace(go.Scatter(x=result_temp.trend.index, y=result_temp.trend, 
                                        mode='lines', name='Trend'), row=2, col=1)
decomposed_fig_temp.add_trace(go.Scatter(x=result_temp.seasonal.index, y=result_temp.seasonal, 
                                        mode='lines', name='Seasonal'), row=3, col=1)
decomposed_fig_temp.add_trace(go.Scatter(x=result_temp.resid.index, y=result_temp.resid, 
                                        mode='lines', name='Residual'), row=4, col=1)

decomposed_fig_temp.update_layout(height=700, title_text="멜버른 기온 데이터 시계열 분해 (가법 모델)", 
                                 showlegend=False)
decomposed_fig_temp.show()


#### 분해 결과 분석

- **원본 데이터(Observed)**: 1981년부터 1990년까지 멜버른의 일일 최저기온 데이터로, 뚜렷한 계절적 변동과 함께 연간 주기적 패턴을 보임
- **추세(Trend)**: 전체 기간에 걸쳐 약간의 상승 경향을 보이며, 1980년대 중반 이후 소폭 증가하는 장기적 추세가 관찰됨
- **계절성(Seasonal)**: 매우 규칙적인 연간 주기를 보이며, 남반구 특성상 12-2월(여름철)에 최고점, 6-8월(겨울철)에 최저점을 나타내는 뚜렷한 계절적 패턴
- **잔차(Residual)**: 추세와 계절성을 제거한 후 남은 불규칙한 변동으로, 대부분 -2°C에서 +2°C 범위 내에서 변동하며 상대적으로 작은 크기의 랜덤한 노이즈 특성을 보임
- **전체적으로 가법 모델이 데이터를 잘 분해하였으며, 계절성 성분이 가장 지배적인 요소임을 확인할 수 있음**


---

## 2. 이동 평균을 이용한 시계열 평활 (Smoothing with Moving Averages)

시계열 데이터의 불규칙한 변동(잔차)을 제거하고 장기적인 추세를 명확하게 파악하기 위해 **평활(Smoothing)** 기법을 사용합니다. 이동 평균은 가장 대표적인 평활 기법 중 하나입니다.

### 2-1. 단순 이동 평균 (Simple Moving Average - SMA)

#### 💡 개념 (Concept)

단순 이동 평균(SMA)은 특정 기간(Window) 동안의 데이터 값들의 산술 평균을 계산하는 방법입니다. 예를 들어, 7일 이동 평균은 오늘을 포함한 최근 7일간의 데이터 평균을 계산하고, 이 과정을 하루씩 이동하며 반복합니다.

-   **장점**: 계산이 간단하고 직관적이며, 데이터의 단기적인 노이즈를 효과적으로 줄여줍니다.
-   **단점**:
    -   과거 데이터에 모두 동일한 가중치를 부여하여 최신 정보의 중요성을 반영하지 못합니다.
    -   항상 실제 데이터의 변화보다 늦게 반응하는 **지연(Lag)**이 발생합니다.

#### 💻 예시 코드 (Example Code)

항공기 탑승객 데이터에 6개월, 12개월 SMA를 적용하여 추세를 비교해 보겠습니다. 12개월 SMA가 단기 변동을 더 많이 제거하여 장기 추세를 더 부드럽게 보여주는 것을 확인할 수 있습니다.

In [ ]:
# 1. 6개월 및 12개월 단순 이동 평균 계산
df_air['6개월 SMA'] = df_air['탑승객 수'].rolling(window=6).mean()
df_air['12개월 SMA'] = df_air['탑승객 수'].rolling(window=12).mean()

# 2. 원본 데이터와 이동 평균선 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['탑승객 수'], mode='lines', name='원본'))
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['6개월 SMA'], mode='lines', name='6개월 SMA'))
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['12개월 SMA'], mode='lines', name='12개월 SMA'))

fig.update_layout(title='항공기 탑승객 수와 단순 이동 평균(SMA)', legend_x=0.01, legend_y=0.99)
fig.show()

#### ✏️ 연습 문제 (Practice Problems)

1.  미국 소매 판매(음식료품) 월별 데이터는 뚜렷한 추세와 계절성을 가집니다. 아래 URL의 데이터를 불러와 `DATE`를 인덱스로, `MRTSSM4453USN` 컬럼을 `소매판매액`으로 설정하여 시계열 데이터를 준비하세요.
    * 데이터 URL: `https://raw.githubusercontent.com/pgroner/Tutorials/master/Time_Series_Analysis/Retail_sales.csv`

2.  준비된 소매 판매 데이터에 대해 6개월, 12개월, 24개월 SMA를 각각 계산하여 데이터프레임에 추가하세요.

3.  원본 데이터와 세 개의 SMA선을 하나의 그래프에 시각화하고, 윈도우 크기(6, 12, 24)가 커질수록 그래프가 어떻게 변하는지(부드러움, 지연 현상) 관찰하고 설명해 보세요.

In [ ]:
# 연습 문제 1, 2, 3번 풀이 공간
# 1. 미국 소매 판매 데이터 불러오기 및 전처리
import pandas as pd
import plotly.graph_objects as go

# 데이터 불러오기
url = 'https://raw.githubusercontent.com/pgroner/Tutorials/master/Time_Series_Analysis/Retail_sales.csv'
df_retail = pd.read_csv(url)

# DATE를 인덱스로 설정하고 datetime 형식으로 변환
df_retail['DATE'] = pd.to_datetime(df_retail['DATE'])
df_retail.set_index('DATE', inplace=True)

# 컬럼명 변경
df_retail.rename(columns={'MRTSSM4453USN': '소매판매액'}, inplace=True)

print("소매 판매 데이터 정보:")
print(df_retail.head())
print(f"데이터 기간: {df_retail.index.min()} ~ {df_retail.index.max()}")

# 2. 6개월, 12개월, 24개월 SMA 계산
df_retail['6개월 SMA'] = df_retail['소매판매액'].rolling(window=6).mean()
df_retail['12개월 SMA'] = df_retail['소매판매액'].rolling(window=12).mean()
df_retail['24개월 SMA'] = df_retail['소매판매액'].rolling(window=24).mean()

# 3. 원본 데이터와 세 개의 SMA선 시각화
fig = go.Figure()

# 원본 데이터
fig.add_trace(go.Scatter(
    x=df_retail.index, 
    y=df_retail['소매판매액'], 
    mode='lines', 
    name='원본 데이터',
    line=dict(color='lightblue', width=1),
    opacity=0.7
))

# 6개월 SMA
fig.add_trace(go.Scatter(
    x=df_retail.index, 
    y=df_retail['6개월 SMA'], 
    mode='lines', 
    name='6개월 SMA',
    line=dict(color='green', width=2)
))

# 12개월 SMA
fig.add_trace(go.Scatter(
    x=df_retail.index, 
    y=df_retail['12개월 SMA'], 
    mode='lines', 
    name='12개월 SMA',
    line=dict(color='orange', width=2)
))

# 24개월 SMA
fig.add_trace(go.Scatter(
    x=df_retail.index, 
    y=df_retail['24개월 SMA'], 
    mode='lines', 
    name='24개월 SMA',
    line=dict(color='red', width=2)
))

fig.update_layout(
    title='미국 소매 판매액과 단순 이동 평균(SMA) 비교',
    xaxis_title='날짜',
    yaxis_title='소매 판매액',
    legend=dict(x=0.01, y=0.99),
    width=900,
    height=500
)

fig.show()

# 윈도우 크기에 따른 변화 분석
print("\n=== 윈도우 크기에 따른 SMA 특성 분석 ===")
print("1. 부드러움(Smoothness):")
print("   - 6개월 SMA: 원본 데이터의 단기 변동을 일부 제거하지만 여전히 변동성이 존재")
print("   - 12개월 SMA: 계절적 변동을 상당히 제거하여 더 부드러운 추세선 형성")
print("   - 24개월 SMA: 가장 부드러운 추세선으로 장기적인 트렌드만 명확히 표현")
print()
print("2. 지연 현상(Lag):")
print("   - 윈도우 크기가 클수록 지연이 더 크게 발생")
print("   - 6개월 SMA: 상대적으로 빠른 반응, 작은 지연")
print("   - 12개월 SMA: 중간 정도의 지연")
print("   - 24개월 SMA: 가장 큰 지연, 실제 변화보다 늦게 반응")
print()
print("3. 추세 파악:")
print("   - 단기 추세: 6개월 SMA가 적합")
print("   - 중기 추세: 12개월 SMA가 적합")
print("   - 장기 추세: 24개월 SMA가 적합")

### 2-2. 지수 가중 이동 평균 (Exponentially Weighted Moving Average - EWMA)

#### 💡 개념 (Concept)

지수 가중 이동 평균(EWMA)은 SMA의 단점을 보완하기 위해 고안되었습니다. 모든 과거 데이터를 동일한 가중치로 평균 내는 SMA와 달리, EWMA는 **최신 데이터에 더 높은 가중치를 부여**하고 과거 데이터일수록 가중치를 지수적으로 감소시킵니다. 이로 인해 최신 데이터의 변화에 더 민감하게 반응하면서도 전체적인 추세를 부드럽게 표현할 수 있습니다.

$EWMA_t = \alpha \cdot Y_t + (1 - \alpha) \cdot EWMA_{t-1}$

* $Y_t$: 시간 $t$의 실제 값
* $EWMA_{t-1}$: 이전 시점의 EWMA 값
* $\alpha$: 평활 계수(Smoothing Parameter). 0과 1 사이의 값을 가지며, $\alpha$가 클수록 최신 데이터에 더 많은 가중치를 부여합니다.

`pandas`에서는 `span`이라는 파라미터를 통해 $\alpha$를 간접적으로 설정할 수 있습니다 ($\alpha = 2 / (span + 1)$). `span`은 대략 SMA의 윈도우 크기와 유사한 의미를 가집니다.

#### 💻 예시 코드 (Example Code)

항공기 탑승객 데이터에 12개월 SMA와 12-span EWMA를 함께 그려 비교해 보겠습니다. EWMA가 SMA보다 지연이 적고 원본 데이터를 더 잘 따라가는 것을 볼 수 있습니다.

In [ ]:
# 1. 12-span EWMA 계산
df_air['12-span EWMA'] = df_air['탑승객 수'].ewm(span=12, adjust=False).mean()

# 2. 원본, SMA, EWMA 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['탑승객 수'], mode='lines', name='원본', opacity=0.7))
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['12개월 SMA'], mode='lines', name='12개월 SMA', line=dict(dash='dash')))
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['12-span EWMA'], mode='lines', name='12-span EWMA', line=dict(color='red')))

fig.update_layout(title='SMA와 EWMA 비교', legend_x=0.01, legend_y=0.99)
fig.show()

#### ✏️ 연습 문제 (Practice Problems)

1.  이전 문제에서 사용한 미국 소매 판매 데이터를 다시 사용합니다.
2.  `span` 값을 각각 6, 12, 24로 설정하여 세 가지 EWMA를 계산하고 데이터프레임에 추가하세요.
3.  원본 데이터와 세 개의 EWMA선을 하나의 그래프에 시각화하고, `span` 값에 따른 변화를 관찰해 보세요.
4.  (심화) 12개월 SMA와 12-span EWMA를 비교했을 때, 어떤 선이 소매 판매 데이터의 추세를 더 잘 반영한다고 생각하는지, 그 이유를 설명해 보세요.

In [ ]:
# 연습 문제 1, 2, 3, 4번 풀이 공간
# 1. 미국 소매 판매 데이터에 span 값 6, 12, 24로 EWMA 계산
df_retail['6-span EWMA'] = df_retail['소매 판매'].ewm(span=6, adjust=False).mean()
df_retail['12-span EWMA'] = df_retail['소매 판매'].ewm(span=12, adjust=False).mean()
df_retail['24-span EWMA'] = df_retail['소매 판매'].ewm(span=24, adjust=False).mean()

# 2. 원본 데이터와 세 개의 EWMA 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_retail.index, y=df_retail['소매 판매'], mode='lines', name='원본', opacity=0.7))
fig.add_trace(go.Scatter(x=df_retail.index, y=df_retail['6-span EWMA'], mode='lines', name='6-span EWMA', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_retail.index, y=df_retail['12-span EWMA'], mode='lines', name='12-span EWMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df_retail.index, y=df_retail['24-span EWMA'], mode='lines', name='24-span EWMA', line=dict(color='green')))

fig.update_layout(
    title='미국 소매 판매 데이터: 다양한 span 값의 EWMA 비교',
    xaxis_title='날짜',
    yaxis_title='소매 판매 (백만 달러)',
    legend_x=0.01,
    legend_y=0.99
)
fig.show()

# 3. 12개월 SMA와 12-span EWMA 비교
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=df_retail.index, y=df_retail['소매 판매'], mode='lines', name='원본', opacity=0.7))
fig2.add_trace(go.Scatter(x=df_retail.index, y=df_retail['12개월 SMA'], mode='lines', name='12개월 SMA', line=dict(dash='dash', color='blue')))
fig2.add_trace(go.Scatter(x=df_retail.index, y=df_retail['12-span EWMA'], mode='lines', name='12-span EWMA', line=dict(color='red')))

fig2.update_layout(
    title='미국 소매 판매 데이터: SMA vs EWMA 비교',
    xaxis_title='날짜',
    yaxis_title='소매 판매 (백만 달러)',
    legend_x=0.01,
    legend_y=0.99
)
fig2.show()

# 4. 심화 분석: SMA와 EWMA 비교 결과 해석
print("📊 SMA vs EWMA 비교 분석:")
print("1. 지연(Lag): 12-span EWMA가 12개월 SMA보다 지연이 적어 최신 데이터 변화에 더 빠르게 반응합니다.")
print("2. 민감도: EWMA는 최신 데이터에 더 높은 가중치를 부여하여 급격한 변화를 더 잘 포착합니다.")
print("3. 평활성: SMA는 더 부드러운 곡선을 만들지만, EWMA는 원본 데이터의 패턴을 더 잘 따라갑니다.")
print("4. 추세 반영: 소매 판매 데이터의 경우, EWMA가 상승 추세와 하락 추세를 더 빠르게 감지하여")
print("   실제 비즈니스 의사결정에 더 유용한 정보를 제공한다고 판단됩니다.")



---

## 3. 홀트-윈터스 기법을 이용한 예측 (Forecasting with Holt-Winters)

### 💡 개념 (Concept)

홀트-윈터스(Holt-Winters) 기법은 EWMA를 확장하여 **추세(Trend)와 계절성(Seasonality)을 명시적으로 모델링**하고 예측하는 강력한 시계열 분석 방법입니다. **삼중 지수 평활(Triple Exponential Smoothing)** 이라고도 불립니다.

-   **단순 지수 평활 (Simple)**: 수준(Level)만 고려합니다. 추세나 계절성이 없는 데이터에 적합합니다.
-   **이중 지수 평활 (Double / Holt's Method)**: 수준과 **추세**를 함께 고려합니다. 추세는 있지만 계절성은 없는 데이터에 적합합니다.
-   **삼중 지수 평활 (Triple / Holt-Winters Method)**: 수준, 추세, **계절성**을 모두 고려합니다. 가장 복잡하고 많은 시계열 데이터에 적용할 수 있습니다.

각 요소(추세, 계절성)가 시간에 따라 어떻게 변하는지에 따라 **가법(Additive)** 모델과 **승법(Multiplicative)** 모델로 나뉩니다.

-   `trend='add'`: 추세가 시간에 따라 일정하게 증가/감소할 때.
-   `trend='mul'`: 추세가 시간에 따라 지수적으로(비율로) 증가/감소할 때.
-   `seasonal='add'`: 계절성의 변동폭이 데이터의 크기와 무관하게 일정할 때.
-   `seasonal='mul'`: 계절성의 변동폭이 데이터의 크기에 비례하여 변할 때.

### 💻 예시 코드 (Example Code)

항공기 탑승객 데이터는 뚜렷한 추세와 계절성을 모두 가지고 있으므로, 삼중 지수 평활(Holt-Winters)이 적합합니다. 추세와 계절성 모두 승법 모델을 적용하여 모델을 학습시키고, 학습된 패턴을 확인해 보겠습니다.

In [ ]:
# 1. 모델 정의 및 학습
# freq='MS'는 월의 시작일(Month Start) 빈도를 의미합니다.
# 데이터의 추세와 계절성이 승법적이므로 trend='mul', seasonal='mul'로 설정
model = ExponentialSmoothing(
    df_air['탑승객 수'],
    trend='mul',
    seasonal='mul',
    seasonal_periods=12,  # 1년 주기의 계절성
    freq='MS'
)
fit = model.fit()

# 2. 모델이 학습한 값(Fitted values)을 데이터프레임에 추가
df_air['Holt-Winters'] = fit.fittedvalues

# 3. 원본과 모델 학습 결과 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['탑승객 수'], mode='lines', name='원본'))
fig.add_trace(go.Scatter(x=df_air.index, y=df_air['Holt-Winters'], mode='lines', name='Holt-Winters 예측', line=dict(color='red', dash='dash')))

fig.update_layout(title='Holt-Winters 삼중 지수 평활 모델', legend_x=0.01, legend_y=0.99)
fig.show()

# 4. 향후 24개월(2년) 예측
forecast = fit.forecast(24)

# 5. 예측 결과 시각화
fig_forecast = go.Figure()
fig_forecast.add_trace(go.Scatter(x=df_air.index, y=df_air['탑승객 수'], mode='lines', name='원본 데이터'))
fig_forecast.add_trace(go.Scatter(x=forecast.index, y=forecast, mode='lines', name='미래 예측', line=dict(color='green')))

fig_forecast.update_layout(title='Holt-Winters를 이용한 미래 탑승객 수 예측', legend_x=0.01, legend_y=0.99)
fig_forecast.show()

### ✏️ 연습 문제 (Practice Problems)

아래 URL은 3년간의 월별 샴푸 판매량 데이터입니다. 이 데이터를 사용하여 홀트-윈터스 모델을 학습하고 미래 판매량을 예측해 봅시다.

-   데이터 URL: `https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv`

1.  데이터를 로드하고, `Month` 컬럼을 이용해 시계열 인덱스를 만드세요. (힌트: `pd.date_range`와 `pd.to_datetime`을 조합하여 '1-01', '1-02' 같은 형식을 '2001-01-01', '2001-02-01' 등으로 변환할 수 있습니다.)

2.  데이터를 학습용(처음 24개월)과 테스트용(마지막 12개월)으로 분리하세요.

3.  학습용 데이터를 사용하여 홀트-윈터스 모델을 학습시키세요. 샴푸 판매 데이터는 추세는 완만하고 계절성의 변동폭은 커 보이지 않으므로, `trend='add'`, `seasonal='add'`, `seasonal_periods=12`로 설정해 보세요.

4.  학습된 모델을 사용하여 향후 12개월의 판매량을 예측하세요.

5.  하나의 그래프에 `(1)전체 원본 데이터`, `(2)모델이 학습한 값(fittedvalues)`, `(3)미래 예측치`를 모두 그려서 모델이 데이터를 얼마나 잘 설명하고 예측하는지 시각적으로 평가해 보세요.

In [ ]:
# 연습 문제 1, 2, 3, 4, 5번 풀이 공간
# 1. 데이터 로드 및 시계열 인덱스 생성
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# 데이터 로드
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/shampoo.csv'
df_shampoo = pd.read_csv(url)

# Month 컬럼을 날짜 형식으로 변환
# '1-01' 형식을 '2001-01-01' 형식으로 변환
df_shampoo['Month'] = pd.to_datetime('200' + df_shampoo['Month'], format='%Y-%m')
df_shampoo.set_index('Month', inplace=True)

print("샴푸 판매 데이터 정보:")
print(df_shampoo.head())
print(f"데이터 크기: {df_shampoo.shape}")

# 2. 데이터 분리 (학습용 24개월, 테스트용 12개월)
train_data = df_shampoo.iloc[:24]  # 처음 24개월
test_data = df_shampoo.iloc[24:]   # 마지막 12개월

print(f"\n학습 데이터 크기: {train_data.shape}")
print(f"테스트 데이터 크기: {test_data.shape}")

# 3. 홀트-윈터스 모델 학습
model = ExponentialSmoothing(
    train_data['Sales'],
    trend='add',
    seasonal='add',
    seasonal_periods=12,
    freq='MS'
)
fit = model.fit()

# 4. 향후 12개월 예측
forecast = fit.forecast(12)

# 학습된 모델의 fitted values 추가
train_data_copy = train_data.copy()
train_data_copy['Fitted'] = fit.fittedvalues

# 5. 전체 결과 시각화
fig = go.Figure()

# (1) 전체 원본 데이터
fig.add_trace(go.Scatter(
    x=df_shampoo.index, 
    y=df_shampoo['Sales'], 
    mode='lines+markers', 
    name='원본 데이터',
    line=dict(color='blue')
))

# (2) 모델이 학습한 값 (fitted values)
fig.add_trace(go.Scatter(
    x=train_data_copy.index, 
    y=train_data_copy['Fitted'], 
    mode='lines', 
    name='Holt-Winters 학습값',
    line=dict(color='red', dash='dash')
))

# (3) 미래 예측치
fig.add_trace(go.Scatter(
    x=forecast.index, 
    y=forecast, 
    mode='lines+markers', 
    name='미래 예측',
    line=dict(color='green', width=3)
))

# 학습/테스트 구분선 추가
fig.add_vline(
    x=train_data.index[-1], 
    line_dash="dot", 
    line_color="gray",
    annotation_text="학습/예측 구분선"
)

fig.update_layout(
    title='홀트-윈터스 모델을 이용한 샴푸 판매량 예측',
    xaxis_title='날짜',
    yaxis_title='판매량',
    legend=dict(x=0.01, y=0.99),
    hovermode='x unified'
)

fig.show()

# 예측 성능 평가 (실제 테스트 데이터와 비교)
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(test_data['Sales'], forecast)
rmse = np.sqrt(mean_squared_error(test_data['Sales'], forecast))

print(f"\n예측 성능 평가:")
print(f"MAE (평균 절대 오차): {mae:.2f}")
print(f"RMSE (평균 제곱근 오차): {rmse:.2f}")

# 예측값과 실제값 비교 테이블
comparison_df = pd.DataFrame({
    '실제값': test_data['Sales'].values,
    '예측값': forecast.values,
    '오차': test_data['Sales'].values - forecast.values
})
comparison_df.index = test_data.index
print(f"\n예측값 vs 실제값 비교:")
print(comparison_df.round(2))

